<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/Review_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pickle


def readTxt(filename):
  print(filename)
  lines = []
  with open(filename,encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()
  
  return lines

neg = readTxt('neg_comment.txt')
pos = readTxt('pos_comment.txt')

y_neg = np.ones((len(neg), 1))
y_pos = np.zeros((len(pos), 1))
y_train = np.concatenate((y_neg, y_pos), axis=0)

neg.extend(pos)
all_text = neg
del neg
del pos
del y_neg
del y_pos
def Fit_Pre(all_text):
  word_vectorizer = TfidfVectorizer(
      sublinear_tf=True,
      strip_accents='unicode',
      encoding='utf-8',
      analyzer='word',
      token_pattern=r'\w{1,}',
      stop_words='english',
      ngram_range=(1, 1),
      norm='l2',
      min_df=0,
      smooth_idf=False,
      max_features=5000)
  word_vectorizer.fit(all_text)
  pickle.dump(word_vectorizer, open('word.sav', 'wb'))
  #train_word_features = word_vectorizer.transform(train)
  char_vectorizer = TfidfVectorizer(
      sublinear_tf=True,
      strip_accents='unicode',
      analyzer='char',
      stop_words='english',
      ngram_range=(2, 6),
      norm='l2',
      min_df=0,
      smooth_idf=False,
      max_features=1500)
  char_vectorizer.fit(all_text)
  pickle.dump(char_vectorizer, open('char.sav', 'wb'))
  #train_char_features = char_vectorizer.transform(train)
  #train_features = hstack([train_char_features, train_word_features])
  #return train_features
Fit_Pre(all_text)
def preprocessing(data):
  word = pickle.load(open('word.sav', 'rb'))
  char = pickle.load(open('char.sav', 'rb'))
  word_features = word.transform(data)
  char_features = char.transform(data)
  features = hstack([char_features, word_features])
  return features
train_features = preprocessing(all_text)

neg_comment.txt
pos_comment.txt


In [0]:
X = train_features

y = y_train
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

def LogReg(X_train, y_train):
  log = LogisticRegression(C=13.0)
  log.fit(X_train, y_train)
  return log

t = time.time()
print("Fitting...")
model_logreg = LogReg(X_train, y_train)
print("Done!")
print("Time to train with LogisticRegression: ", time.time()-t)


Fitting...


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Done!
Time to train with LogisticRegression:  0.6392419338226318


In [5]:
pred_train_logreg = model_logreg.predict(X_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, pred_train_logreg)))

pred_test_logreg = model_logreg.predict(X_val)
print('Testing accuracy is {}'.format(accuracy_score(y_val, pred_test_logreg)))


Training accuracy is 0.9987109249113761
Testing accuracy is 0.9800193361263294


In [6]:
data = list()
data.append('I guess I should have realized a smaller screen wouldn’t fit our needs so it’s reslly not a fault of Amazon.')
data_test = preprocessing(data)

print(model_logreg.predict(data_test))

[1.]


## Support Vector Machine

In [0]:
from sklearn.svm import SVC

svm = SVC(C=13.0)
svm.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:

print('Training accuracy is {}'.format(accuracy_score(y_train, svm.predict(X_train))))

print('Testing accuracy is {}'.format(accuracy_score(y_val, svm.predict(X_val))))

In [0]:
print(data)

['I guess I should have realized a smaller screen wouldn’t fit our needs so it’s reslly not a fault of Amazon.']
